In [ ]:
pip install transformers datasets evaluate rouge_score pandas tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8b36895a223c4792968e3b072307870ba62502160ab8e6df11f0c2efc95777f9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's depen

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler
)
from torch.optim import AdamW
import evaluate
from tqdm import tqdm
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Models
TEACHER_MODEL = "openai-community/gpt2-xl"
STUDENT_MODEL = "openai-community/gpt2-medium"


# Hyperparameters
MAX_LENGTH = 256
BATCH_SIZE = 1
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
TEMPERATURE = 2.0
ALPHA = 0.5

# Load SQuAD subset
def load_squad_sample(sample_size=100):
    from datasets import load_dataset
    return load_dataset("squad", split=f"train[:{sample_size}]")

# Dataset class
class KnowledgeDistillationDataset(Dataset):
    def __init__(self, dataset, teacher_tokenizer, student_tokenizer, max_length):
        self.dataset = dataset
        self.teacher_tokenizer = teacher_tokenizer
        self.student_tokenizer = student_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item["question"]
        context = item["context"]
        answer = item["answers"]["text"][0] if item["answers"]["text"] else ""

        #  Better prompt structure
        prompt = f"Answer the following question based on the context.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"

        teacher_inputs = self.teacher_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_inputs = self.student_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        teacher_labels = self.teacher_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_labels = self.student_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "teacher_input_ids": teacher_inputs.input_ids.squeeze(),
            "teacher_attention_mask": teacher_inputs.attention_mask.squeeze(),
            "teacher_labels": teacher_labels.input_ids.squeeze(),
            "student_input_ids": student_inputs.input_ids.squeeze(),
            "student_attention_mask": student_inputs.attention_mask.squeeze(),
            "student_labels": student_labels.input_ids.squeeze(),
            "prompt": prompt,
            "answer": answer
        }

# Distillation loss
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha, ignore_index):
    vocab_size = student_logits.size(-1)
    if teacher_logits.size(-1) > vocab_size:
        teacher_logits = teacher_logits[..., :vocab_size]

    soft_targets = F.softmax(teacher_logits / temperature, dim=-1)
    soft_prob = F.log_softmax(student_logits / temperature, dim=-1)
    kd_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean') * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits.view(-1, vocab_size), labels.view(-1), ignore_index=ignore_index)
    return alpha * ce_loss + (1 - alpha) * kd_loss, ce_loss, kd_loss

# Training loop
def train_model(teacher_model, student_model, train_loader, optimizer, scheduler, ignore_index):
    student_model.train()
    teacher_model.eval()
    total_loss = total_ce_loss = total_kd_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        student_input_ids = batch["student_input_ids"].to(device)
        student_attention_mask = batch["student_attention_mask"].to(device)
        student_labels = batch["student_labels"].to(device)
        teacher_input_ids = batch["teacher_input_ids"].to(device)
        teacher_attention_mask = batch["teacher_attention_mask"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(teacher_input_ids, attention_mask=teacher_attention_mask).logits

        student_outputs = student_model(student_input_ids, attention_mask=student_attention_mask)
        student_logits = student_outputs.logits

        loss, ce_loss, kd_loss = distillation_loss(student_logits, teacher_logits, student_labels, TEMPERATURE, ALPHA, ignore_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_kd_loss += kd_loss.item()
        progress_bar.set_postfix({
            "Loss": total_loss / (progress_bar.n + 1),
            "CE": total_ce_loss / (progress_bar.n + 1),
            "KD": total_kd_loss / (progress_bar.n + 1)
        })

# Evaluation
def evaluate_model(model, tokenizer, eval_dataset, name="Model"):
    model.eval()
    squad_metric = evaluate.load("squad")
    rouge_metric = evaluate.load("rouge")
    bleu_metric = evaluate.load("bleu")

    results = []
    loader = DataLoader(eval_dataset, batch_size=1)
    all_preds, all_refs = [], []

    for idx, batch in enumerate(tqdm(loader, desc=f"Evaluating {name}")):
        input_ids = batch[f"{name.lower()}_input_ids"].to(device)
        attention_mask = batch[f"{name.lower()}_attention_mask"].to(device)
        prompt = batch["prompt"][0]
        true_answer = batch["answer"][0]

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=64)
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        generated_answer = prediction.replace(prompt, "").strip()
        reference = true_answer.strip()

        results.append({
            "prompt": prompt,
            "reference": reference,
            "prediction": generated_answer
        })

        all_preds.append(generated_answer)
        all_refs.append(reference)

        squad_metric.add(
            prediction={"id": str(idx), "prediction_text": generated_answer},
            reference={"id": str(idx), "answers": {"text": [reference], "answer_start": [0]}}
        )

    squad_scores = squad_metric.compute()
    rouge_scores = rouge_metric.compute(predictions=all_preds, references=all_refs)
    bleu_scores = bleu_metric.compute(predictions=all_preds, references=[[ref] for ref in all_refs])

    combined = {
        "squad": squad_scores,
        "rouge": rouge_scores,
        "bleu": bleu_scores
    }

    return results, combined

# Print metrics
def print_metrics(name, scores):
    print(f"\n{name} Evaluation:")
    print(f"  SQuAD - EM: {scores['squad']['exact_match']:.2f}, F1: {scores['squad']['f1']:.2f}")
    print(f"  ROUGE:")
    for k, v in scores["rouge"].items():
        print(f"    {k}: {v:.2f}")
    print(f"  BLEU: {scores['bleu']['bleu']:.2f}")

# Main
def main():
    print(" Loading dataset...")
    dataset = load_squad_sample(100)

    print(" Loading teacher model...")
    teacher_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
    teacher_model = AutoModelForCausalLM.from_pretrained(TEACHER_MODEL).to(device)

    print(" Loading student model...")
    student_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
    student_tokenizer.pad_token = student_tokenizer.eos_token
    student_pad_token_id = student_tokenizer.pad_token_id
    student_model = AutoModelForCausalLM.from_pretrained(STUDENT_MODEL).to(device)

    print(" Preparing dataset...")
    kd_dataset = KnowledgeDistillationDataset(dataset, teacher_tokenizer, student_tokenizer, MAX_LENGTH)
    train_size = int(0.8 * len(kd_dataset))
    train_dataset, eval_dataset = random_split(kd_dataset, [train_size, len(kd_dataset) - train_size])
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = AdamW(student_model.parameters(), lr=LEARNING_RATE)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*NUM_EPOCHS)

    for epoch in range(NUM_EPOCHS):
        print(f"\n Epoch {epoch+1}/{NUM_EPOCHS}")
        train_model(teacher_model, student_model, train_loader, optimizer, scheduler, student_pad_token_id)

        out_dir = f"student_checkpoint_epoch_{epoch+1}"
        os.makedirs(out_dir, exist_ok=True)
        student_model.save_pretrained(out_dir)
        student_tokenizer.save_pretrained(out_dir)

    print("\n Evaluating models...")
    student_results, student_scores = evaluate_model(student_model, student_tokenizer, eval_dataset, name="Student")
    teacher_results, teacher_scores = evaluate_model(teacher_model, teacher_tokenizer, eval_dataset, name="Teacher")

    print_metrics("Student", student_scores)
    print_metrics("Teacher", teacher_scores)

    pd.DataFrame({
        "Prompt": [r["prompt"] for r in student_results],
        "True Answer": [r["reference"] for r in student_results],
        "Student Prediction": [r["prediction"] for r in student_results],
        "Teacher Prediction": [r["prediction"] for r in teacher_results]
    }).to_csv("distillation_eval_results.csv", index=False)

    print("\n Results saved to distillation_eval_results.csv")

if __name__ == "__main__":
    main()


Using device: cuda
 Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

...


 Epoch 3/3


Training: 100%|██████████| 80/80 [00:47<00:00,  1.68it/s, Loss=61.5, CE=9.14, KD=114]



 Evaluating models...


Evaluating Teacher: 100%|██████████| 20/20 [01:00<00:00,  3.03s/it]



Student Evaluation:
  SQuAD - EM: 0.00, F1: 7.11
  ROUGE:
    rouge1: 0.06
    rouge2: 0.03
    rougeL: 0.06
    rougeLsum: 0.06
  BLEU: 0.01

Teacher Evaluation:
  SQuAD - EM: 0.00, F1: 8.52
  ROUGE:
    rouge1: 0.08
    rouge2: 0.05
    rougeL: 0.08
    rougeLsum: 0.08
  BLEU: 0.01

 Results saved to distillation_eval_results.csv
